In [2]:
import tkinter as tk
import cv2
import pytesseract
import re
import csv
import mysql.connector
from PIL import Image
from tkinter import filedialog, messagebox, ttk
import threading
from mysql.connector import Error

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Configuration de la base de données
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': 'ayaDATA2025@',
    'database': 'extraction_factures'
}

def create_database_connection():
    try:
        conn = mysql.connector.connect(**DB_CONFIG)
        return conn
    except Error as e:
        messagebox.showerror("Database Error", f"Erreur de connexion à la base de données: {e}")
        return None

def preprocess_image(image_path):
    try:
        img = cv2.imread(image_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        binarized = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                       cv2.THRESH_BINARY, 11, 2)
        denoised = cv2.medianBlur(binarized, 3)
        return denoised
    except Exception as e:
        raise RuntimeError(f"Erreur de prétraitement d'image: {str(e)}")

def extract_information(text):
    data = {
        'seller': {'name': '', 'address': '', 'tax_id': ''},
        'client': {'name': '', 'address': '', 'tax_id': ''},
        'invoice_number': '',
        'date': '',
        'iban': '',
        'montants': [],
        'vat': 0.0,
        'net_worth': 0.0
    }

    # Extraction des sections Client/Vendeur
    def extract_entity(entity_type):
        section = re.search(fr'{entity_type}[\s:]*\n((?:.*\n)+?)(?=\n\s*\n|$)', text, re.IGNORECASE)
        if section:
            lines = [line.strip() for line in section.group(1).split('\n') if line.strip()]
            return {
                'name': lines[0] if len(lines) > 0 else '',
                'address': ' '.join(lines[1:-1]) if len(lines) > 2 else '',
                'tax_id': re.search(r'(Tax|VAT)[\sID:]*(\b[A-Z0-9]+\b)', lines[-1] if lines else '').group(2) if lines else ''
            }
        return {'name': '', 'address': '', 'tax_id': ''}

    data['seller'] = extract_entity('Seller')
    data['client'] = extract_entity('Client')

    # Extraction des autres champs
    patterns = {
        'invoice_number': r'Invoice[\s#]*No[:\s]*([A-Z0-9-]+)',
        'date': r'\b(\d{2}[/-]\d{2}[/-]\d{4})\b',
        'iban': r'IBAN[\s:]*([A-Z]{2}\d{2}[\sA-Z0-9]{10,30})',
        'vat': r'VAT[\s%:]*([\d.,]+)',
        'net_worth': r'Net[\s-]*Worth[\s:]*([\d.,]+)'
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            value = match.group(1).replace(',', '.').strip()
            if key in ['vat', 'net_worth']:
                data[key] = float(value) if value else 0.0
            else:
                data[key] = value

    # Extraction des montants
    data['montants'] = [float(m.replace(',', '.')) 
                       for m in re.findall(r'\b\d{1,3}(?:[.,\s]\d{3})*(?:[.,]\d{2})\b', text)]
    
    return data

def save_to_database(data):
    try:
        conn = create_database_connection()
        if conn is None:
            return

        cursor = conn.cursor()

        # Insertion des entités
        def upsert_entity(table, entity):
            cursor.execute(f"""
                INSERT INTO {table} (name, address, tax_id)
                VALUES (%s, %s, %s)
                ON DUPLICATE KEY UPDATE
                name = VALUES(name), address = VALUES(address)
            """, (entity['name'], entity['address'], entity['tax_id']))
            return cursor.lastrowid if cursor.lastrowid else cursor.rowcount

        seller_id = upsert_entity('sellers', data['seller'])
        client_id = upsert_entity('clients', data['client'])

        # Insertion de la facture
        cursor.execute("""
            INSERT INTO invoices 
            (invoice_number, date, iban, total, vat, net_worth, client_id, seller_id)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            data['invoice_number'],
            data['date'],
            data['iban'],
            max(data['montants']) if data['montants'] else 0.0,
            data['vat'],
            data['net_worth'],
            client_id,
            seller_id
        ))

        conn.commit()
        cursor.close()
        conn.close()

    except Error as e:
        raise RuntimeError(f"Erreur base de données: {str(e)}")

def save_to_csv(data):
    try:
        with open('facture_data.csv', 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow([
                'Invoice Number', 'Date', 'IBAN',
                'Client Name', 'Client Address', 'Client Tax ID',
                'Seller Name', 'Seller Address', 'Seller Tax ID',
                'Total', 'VAT', 'Net Worth'
            ])
            writer.writerow([
                data['invoice_number'],
                data['date'],
                data['iban'],
                data['client']['name'],
                data['client']['address'],
                data['client']['tax_id'],
                data['seller']['name'],
                data['seller']['address'],
                data['seller']['tax_id'],
                max(data['montants']) if data['montants'] else '',
                data['vat'],
                data['net_worth']
            ])
    except IOError as e:
        raise RuntimeError(f"Erreur d'écriture CSV: {str(e)}")

def analyse_facture(filepath):
    try:
        # Prétraitement et OCR
        processed_img = preprocess_image(filepath)
        text = pytesseract.image_to_string(processed_img)
        text = re.sub(r'Tota[l1i]', 'Total', text, flags=re.IGNORECASE)

        # Extraction des données
        data = extract_information(text)

        # Enregistrement des données
        save_to_database(data)
        save_to_csv(data)

        return data

    except Exception as e:
        raise RuntimeError(f"Erreur d'analyse: {str(e)}")

# Interface graphique
class Application(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Analyseur de Factures Intelligent")
        self.geometry("720x520")
        self.configure_styles()
        self.create_widgets()

    def configure_styles(self):
        self.style = ttk.Style()
        self.style.theme_use('clam')
        self.style.configure('TLabel', font=('Helvetica', 10))
        self.style.configure('TButton', font=('Helvetica', 10, 'bold'))

    def create_widgets(self):
        # Header
        self.header = ttk.Frame(self)
        self.title_label = ttk.Label(self.header, 
                                    text="Analyseur de Factures 2.0", 
                                    font=('Helvetica', 16, 'bold'))
        self.btn_open = ttk.Button(self.header, text="Ouvrir une facture", 
                                 command=self.open_file)
        
        # File info
        self.file_frame = ttk.LabelFrame(self, text="Fichier sélectionné")
        self.file_label = ttk.Label(self.file_frame, text="Aucun fichier sélectionné")
        self.status_label = ttk.Label(self.file_frame, foreground="gray")

        # Results display
        self.results_text = tk.Text(self, wrap=tk.WORD, height=15, 
                                  font=('Consolas', 10), bg='#f0f0f0')

        # Layout
        self.header.pack(pady=10, fill='x')
        self.title_label.pack(side='left', padx=10)
        self.btn_open.pack(side='right', padx=10)
        self.file_frame.pack(fill='x', padx=10, pady=5)
        self.file_label.pack(side='left', padx=5)
        self.status_label.pack(side='right', padx=5)
        self.results_text.pack(fill='both', expand=True, padx=10, pady=5)

    def open_file(self):
        filetypes = [("Images", "*.png;*.jpg;*.jpeg;*.tiff;*.bmp")]
        filepath = filedialog.askopenfilename(title="Sélectionner une facture", filetypes=filetypes)
        if filepath:
            self.status_label.config(text="Analyse en cours...", foreground="blue")
            threading.Thread(target=self.process_file, args=(filepath,), daemon=True).start()

    def process_file(self, filepath):
        try:
            data = analyse_facture(filepath)
            self.after(0, self.update_ui_success, filepath, data)
        except Exception as e:
            self.after(0, self.show_error, str(e))

    def update_ui_success(self, filepath, data):
        self.file_label.config(text=f"Fichier analysé : {filepath}")
        self.status_label.config(text="Analyse réussie", foreground="green")
        
        results = [
            f"Numéro de facture: {data['invoice_number']}",
            f"Date: {data['date']}",
            f"IBAN: {data['iban']}",
            "\nClient:",
            f"Nom: {data['client']['name']}",
            f"Adresse: {data['client']['address']}",
            f"Tax ID: {data['client']['tax_id']}",
            "\nVendeur:",
            f"Nom: {data['seller']['name']}",
            f"Adresse: {data['seller']['address']}",
            f"Tax ID: {data['seller']['tax_id']}",
            "\nMontants:",
            f"Total: {max(data['montants']) if data['montants'] else 'N/A'}",
            f"TVA: {data['vat']}",
            f"Valeur nette: {data['net_worth']}"
        ]
        
        self.results_text.delete(1.0, tk.END)
        self.results_text.insert(tk.END, '\n'.join(results))

    def show_error(self, message):
        messagebox.showerror("Erreur", message)
        self.status_label.config(text="Échec de l'analyse", foreground="red")

if __name__ == "__main__":
    app = Application()
    app.mainloop()

In [7]:
import tkinter as tk
import cv2
import pytesseract
import re
import csv
import mysql.connector
from datetime import datetime
from PIL import Image
from tkinter import filedialog, messagebox, ttk
import threading
from mysql.connector import Error

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': 'ayaDATA2025@',
    'database': 'extraction_factures'
}

# Fonction de connexion à la base de données
def create_connection():
    try:
        return mysql.connector.connect(**DB_CONFIG)
    except Error as e:
        messagebox.showerror("Erreur Base de Données", f"Erreur de connexion : {str(e)}")
        return None

# Prétraitement de l'image
def preprocess_image(image_path):
    try:
        img = cv2.imread(image_path)
        if img is None:
            raise ValueError("Fichier image non valide")
            
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        binarized = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                        cv2.THRESH_BINARY, 11, 2)
        denoised = cv2.medianBlur(binarized, 3)
        return denoised
    except Exception as e:
        raise RuntimeError(f"Erreur de traitement d'image : {str(e)}")

# Extraction des entités (vendeur/client)
def extract_entity(text, entity_type):
    pattern = fr'## {entity_type}:[\s\S]*?(Tax Id:.*?)(?:\n##|\Z)'
    match = re.search(pattern, text, re.IGNORECASE)
    
    entity_info = {'name': '', 'address': '', 'tax_id': ''}
    if match:
        content = match.group(0)
        
        # Extraction du nom
        name_match = re.search(r'## '+entity_type+':\s*\n([^\n]+)', content)
        if name_match:
            entity_info['name'] = name_match.group(1).strip()
        
        # Extraction de l'adresse
        address_match = re.search(r'(?<=## '+entity_type+':\s*\n)(.*?)(?=Tax Id:)', 
                                content, re.DOTALL)
        if address_match:
            address_lines = [line.strip() 
                           for line in address_match.group(1).split('\n') 
                           if line.strip()]
            entity_info['address'] = '\n'.join(address_lines)
        
        # Extraction du Tax ID
        tax_match = re.search(r'Tax Id:\s*([\d-]+)', content)
        if tax_match:
            entity_info['tax_id'] = tax_match.group(1)
    
    return entity_info

# Extraction des informations financières avec gestion améliorée des dates
def extract_financial_data(text):
    data = {
        'invoice_number': '',
        'date': None,
        'iban': '',
        'total_net': 0.0,
        'total_vat': 0.0,
        'total_gross': 0.0
    }
    
    # Extraction numéro de facture
    inv_match = re.search(r'# Invoice no:\s*(\S+)', text)
    if inv_match:
        data['invoice_number'] = inv_match.group(1).strip()

    # Extraction et conversion de la date
    date_match = re.search(r'Date of issue:\s*(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})', text)
    if date_match:
        raw_date = date_match.group(1)
        date_formats = [
            '%m/%d/%Y', '%d/%m/%Y', 
            '%m-%d-%Y', '%d-%m-%Y',
            '%Y/%m/%d', '%Y-%m-%d'
        ]
        
        for fmt in date_formats:
            try:
                parsed_date = datetime.strptime(raw_date, fmt)
                data['date'] = parsed_date.strftime('%Y-%m-%d')
                break
            except ValueError:
                continue

    # Extraction IBAN
    iban_match = re.search(r'IBAN:\s*([A-Z]{2}\d{2}[\sA-Z0-9]{10,30})', text)
    if iban_match:
        data['iban'] = iban_match.group(1).replace(' ', '')

    # Extraction des totaux avec gestion des séparateurs
    total_pattern = r'Total\s*\|\s*([\d\s.,]+)\s*\|\s*([\d\s.,]+)\s*\|\s*([\d\s.,]+)'
    totals_match = re.search(total_pattern, text)
    
    if totals_match:
        try:
            data['total_net'] = float(totals_match.group(1)
                .replace(' ', '')
                .replace(',', '.'))
            data['total_vat'] = float(totals_match.group(2)
                .replace(' ', '')
                .replace(',', '.'))
            data['total_gross'] = float(totals_match.group(3)
                .replace(' ', '')
                .replace(',', '.'))
        except ValueError:
            pass

    return data

# Traitement principal de la facture
def process_invoice(filepath):
    try:
        # OCR et prétraitement
        processed_img = preprocess_image(filepath)
        text = pytesseract.image_to_string(processed_img)
        
        # Extraction des données
        seller = extract_entity(text, 'Seller')
        client = extract_entity(text, 'Client')
        financials = extract_financial_data(text)
        
        if not financials['date']:
            raise ValueError("Format de date non reconnu")

        # Enregistrement en base
        conn = create_connection()
        if conn:
            cursor = conn.cursor()
            
            # Insertion des entités avec gestion de conflit
            cursor.execute("""
                INSERT INTO sellers (name, address, tax_id)
                VALUES (%s, %s, %s)
                ON DUPLICATE KEY UPDATE 
                name=VALUES(name), address=VALUES(address)
            """, (seller['name'], seller['address'], seller['tax_id']))
            
            cursor.execute("""
                INSERT INTO clients (name, address, tax_id)
                VALUES (%s, %s, %s)
                ON DUPLICATE KEY UPDATE 
                name=VALUES(name), address=VALUES(address)
            """, (client['name'], client['address'], client['tax_id']))
            
            # Insertion de la facture
            cursor.execute("""
                INSERT INTO invoices (
                    invoice_number, date, iban, 
                    total_net, total_vat, total_gross,
                    seller_id, client_id
                ) VALUES (%s, %s, %s, %s, %s, %s, 
                (SELECT id FROM sellers WHERE tax_id = %s), 
                (SELECT id FROM clients WHERE tax_id = %s))
            """, (
                financials['invoice_number'],
                financials['date'],
                financials['iban'],
                financials['total_net'],
                financials['total_vat'],
                financials['total_gross'],
                seller['tax_id'],
                client['tax_id']
            ))
            
            conn.commit()
            cursor.close()
            conn.close()
        
        return {
            'seller': seller,
            'client': client,
            'financials': financials
        }
        
    except Exception as e:
        raise RuntimeError(f"Erreur de traitement : {str(e)}")

# Interface graphique
class InvoiceAnalyzerApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Analyseur de Factures Pro")
        self.geometry("800x600")
        self._create_widgets()
        self._configure_styles()
    
    def _configure_styles(self):
        self.style = ttk.Style()
        self.style.theme_use('clam')
        self.style.configure('TLabel', font=('Helvetica', 10))
        self.style.configure('TButton', font=('Helvetica', 10, 'bold'))
    
    def _create_widgets(self):
        # Conteneur principal
        main_frame = ttk.Frame(self)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # En-tête
        header_frame = ttk.Frame(main_frame)
        header_frame.pack(fill=tk.X, pady=5)
        
        ttk.Button(header_frame, 
                 text="Ouvrir Facture", 
                 command=self._open_file).pack(side=tk.LEFT)
        
        self.status_label = ttk.Label(header_frame, foreground="gray")
        self.status_label.pack(side=tk.RIGHT, padx=10)
        
        # Affichage des résultats
        self.results_text = tk.Text(main_frame, 
                                  wrap=tk.WORD, 
                                  font=('Consolas', 10),
                                  bg='#f0f0f0')
        self.results_text.pack(fill=tk.BOTH, expand=True)
    
    def _open_file(self):
        filetypes = [("Images", "*.png;*.jpg;*.jpeg;*.tiff;*.bmp")]
        filepath = filedialog.askopenfilename(title="Sélectionner une facture", 
                                            filetypes=filetypes)
        if filepath:
            self.status_label.config(text="Analyse en cours...", foreground="blue")
            threading.Thread(target=self._process_file, 
                           args=(filepath,), 
                           daemon=True).start()
    
    def _process_file(self, filepath):
        try:
            result = process_invoice(filepath)
            self.after(0, self._show_results, filepath, result)
        except Exception as e:
            self.after(0, self._show_error, str(e))
    
    def _show_results(self, filepath, result):
        output = [
            "=== FACTURE ANALYSÉE ===",
            f"Fichier: {filepath}",
            "\n=== VENDEUR ===",
            f"Nom: {result['seller']['name']}",
            f"Adresse:\n{result['seller']['address']}",
            f"Tax ID: {result['seller']['tax_id']}",
            "\n=== CLIENT ===",
            f"Nom: {result['client']['name']}",
            f"Adresse:\n{result['client']['address']}",
            f"Tax ID: {result['client']['tax_id']}",
            "\n=== INFORMATIONS FINANCIÈRES ===",
            f"Numéro: {result['financials']['invoice_number']}",
            f"Date: {result['financials']['date']}",
            f"IBAN: {result['financials']['iban']}",
            f"Total Net: {result['financials']['total_net']:.2f}€",
            f"TVA: {result['financials']['total_vat']:.2f}€",
            f"Total TTC: {result['financials']['total_gross']:.2f}€"
        ]
        
        self.results_text.delete(1.0, tk.END)
        self.results_text.insert(tk.END, '\n'.join(output))
        self.status_label.config(text="Analyse réussie", foreground="green")
    
    def _show_error(self, message):
        messagebox.showerror("Erreur", message)
        self.status_label.config(text="Échec de l'analyse", foreground="red")
        self.results_text.delete(1.0, tk.END)

# Script de création de base de données
def create_database():
    conn = create_connection()
    if conn:
        cursor = conn.cursor()
        
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS clients (
                id INT AUTO_INCREMENT PRIMARY KEY,
                name VARCHAR(255) NOT NULL,
                address TEXT,
                tax_id VARCHAR(255) UNIQUE
            )
        """)
        
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS sellers (
                id INT AUTO_INCREMENT PRIMARY KEY,
                name VARCHAR(255) NOT NULL,
                address TEXT,
                tax_id VARCHAR(255) UNIQUE
            )
        """)
        
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS invoices (
                id INT AUTO_INCREMENT PRIMARY KEY,
                invoice_number VARCHAR(255),
                date DATE,
                iban VARCHAR(34),
                total_net DECIMAL(10,2),
                total_vat DECIMAL(10,2),
                total_gross DECIMAL(10,2),
                seller_id INT,
                client_id INT,
                FOREIGN KEY (seller_id) REFERENCES sellers(id),
                FOREIGN KEY (client_id) REFERENCES clients(id)
            )
        """)
        
        conn.commit()
        cursor.close()
        conn.close()

if __name__ == "__main__":
    create_database()
    app = InvoiceAnalyzerApp()
    app.mainloop()

<>:55: SyntaxWarning: invalid escape sequence '\s'
<>:60: SyntaxWarning: invalid escape sequence '\s'
<>:55: SyntaxWarning: invalid escape sequence '\s'
<>:60: SyntaxWarning: invalid escape sequence '\s'
C:\Users\L13\AppData\Local\Temp\ipykernel_1260\385897315.py:55: SyntaxWarning: invalid escape sequence '\s'
  name_match = re.search(r'## '+entity_type+':\s*\n([^\n]+)', content)
C:\Users\L13\AppData\Local\Temp\ipykernel_1260\385897315.py:60: SyntaxWarning: invalid escape sequence '\s'
  address_match = re.search(r'(?<=## '+entity_type+':\s*\n)(.*?)(?=Tax Id:)',


In [8]:
import tkinter as tk
import cv2
import pytesseract
import re
import mysql.connector
from datetime import datetime
from tkinter import filedialog, messagebox, ttk
import threading
from mysql.connector import Error

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': 'ayaDATA2025@',
    'database': 'extraction_factures'
}

# Fonctions de prétraitement d'image améliorées
def preprocess_image(image_path):
    try:
        img = cv2.imread(image_path)
        if img is None:
            raise ValueError("Impossible de lire le fichier image")

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        denoised = cv2.fastNlMeansDenoising(gray, h=10)
        _, binarized = cv2.threshold(denoised, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        return binarized
    except Exception as e:
        raise RuntimeError(f"Erreur de prétraitement d'image : {str(e)}")

# Fonctions d'extraction optimisées
def extract_entity(text, entity_type):
    pattern = fr'(?s)## {entity_type}:[\s]*\n(.*?)(?=\n##|\nTax Id:|\nIBAN:|\Z)'
    match = re.search(pattern, text, re.IGNORECASE)
    
    entity_info = {'name': '', 'address': '', 'tax_id': ''}
    
    if match:
        content = match.group(1).strip()
        lines = [line.strip() for line in content.split('\n') if line.strip()]
        
        if lines:
            # Extraction du nom (première ligne non vide)
            entity_info['name'] = lines[0]
            
            # Extraction de l'adresse (lignes suivantes jusqu'au Tax ID)
            address_lines = []
            for line in lines[1:]:
                if re.search(r'[a-zA-Z0-9]', line) and not line.startswith('Tax Id:'):
                    address_lines.append(line)
            entity_info['address'] = '\n'.join(address_lines)
    
    # Extraction Tax ID séparée
    tax_pattern = fr'Tax Id:\s*([\d-]+)'
    tax_match = re.search(tax_pattern, text, re.IGNORECASE)
    if tax_match:
        entity_info['tax_id'] = tax_match.group(1)
    
    return entity_info

def extract_financial_data(text):
    data = {
        'invoice_number': '',
        'date': None,
        'iban': '',
        'total_net': 0.0,
        'total_vat': 0.0,
        'total_gross': 0.0,
        'items': []
    }

    # Extraction numéro de facture
    inv_match = re.search(r'# Invoice no:\s*([A-Z0-9-]+)', text)
    if inv_match:
        data['invoice_number'] = inv_match.group(1)

    # Extraction et conversion de la date
    date_match = re.search(r'Date of issue:\s*(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})', text)
    if date_match:
        raw_date = date_match.group(1)
        for fmt in ('%m/%d/%Y', '%d/%m/%Y', '%Y/%m/%d', '%m-%d-%Y', '%d-%m-%Y'):
            try:
                data['date'] = datetime.strptime(raw_date, fmt).strftime('%Y-%m-%d')
                break
            except ValueError:
                continue

    # Extraction IBAN
    iban_match = re.search(r'IBAN:\s*([A-Z]{2}\d{2}[\sA-Z0-9]{10,30})', text)
    if iban_match:
        data['iban'] = iban_match.group(1).replace(' ', '')

    # Extraction des articles
    items_section = re.search(r'## ITEMS[\s\S]*?## SUMMARY', text)
    if items_section:
        items_text = items_section.group(0)
        item_pattern = r'(\d+\.)\s+(.*?)\s+(\d+,\d+)\s+each\s+([\d\s.,]+)\s+([\d\s.,]+)\s+(\d+%)\s+([\d\s.,]+)'
        items = re.findall(item_pattern, items_text, re.DOTALL)
        
        for item in items:
            data['items'].append({
                'number': item[0].strip('.'),
                'description': item[1].strip(),
                'quantity': float(item[2].replace(',', '.')),
                'unit_price': parse_amount(item[3]),
                'net_price': parse_amount(item[4]),
                'vat_percent': float(item[5].strip('%')),
                'gross_price': parse_amount(item[6])
            })

    # Extraction des totaux
    total_pattern = r'Total\s*\|\s*([\d\s.,]+)\s*\|\s*([\d\s.,]+)\s*\|\s*([\d\s.,]+)'
    totals_match = re.search(total_pattern, text.replace('\n', ''))
    if totals_match:
        data['total_net'] = parse_amount(totals_match.group(1))
        data['total_vat'] = parse_amount(totals_match.group(2))
        data['total_gross'] = parse_amount(totals_match.group(3))

    return data

def parse_amount(amount_str):
    return float(re.sub(r'[^\d,]', '', amount_str).replace(',', '.'))

# Connexion et gestion de la base de données
def create_connection():
    try:
        return mysql.connector.connect(**DB_CONFIG)
    except Error as e:
        messagebox.showerror("Erreur Base de Données", f"Erreur de connexion : {str(e)}")
        return None

def save_to_database(data):
    conn = create_connection()
    if not conn:
        return

    try:
        cursor = conn.cursor()

        # Insertion vendeur
        cursor.execute("""
            INSERT INTO sellers (name, address, tax_id)
            VALUES (%s, %s, %s)
            ON DUPLICATE KEY UPDATE name=VALUES(name), address=VALUES(address)
        """, (data['seller']['name'], data['seller']['address'], data['seller']['tax_id']))

        # Insertion client
        cursor.execute("""
            INSERT INTO clients (name, address, tax_id)
            VALUES (%s, %s, %s)
            ON DUPLICATE KEY UPDATE name=VALUES(name), address=VALUES(address)
        """, (data['client']['name'], data['client']['address'], data['client']['tax_id']))

        # Insertion facture
        cursor.execute("""
            INSERT INTO invoices (
                invoice_number, date, iban,
                total_net, total_vat, total_gross,
                seller_id, client_id
            ) VALUES (
                %s, %s, %s,
                %s, %s, %s,
                (SELECT id FROM sellers WHERE tax_id = %s),
                (SELECT id FROM clients WHERE tax_id = %s)
            )
        """, (
            data['invoice_number'],
            data['date'],
            data['iban'],
            data['total_net'],
            data['total_vat'],
            data['total_gross'],
            data['seller']['tax_id'],
            data['client']['tax_id']
        ))

        # Insertion des articles
        invoice_id = cursor.lastrowid
        for item in data['items']:
            cursor.execute("""
                INSERT INTO items (
                    invoice_id, item_number, description,
                    quantity, unit_price, net_price,
                    vat_percent, gross_price
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """, (
                invoice_id,
                item['number'],
                item['description'],
                item['quantity'],
                item['unit_price'],
                item['net_price'],
                item['vat_percent'],
                item['gross_price']
            ))

        conn.commit()
    except Error as e:
        conn.rollback()
        raise RuntimeError(f"Erreur base de données : {str(e)}")
    finally:
        cursor.close()
        conn.close()

# Interface graphique
class InvoiceAnalyzerApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Analyseur de Factures Pro")
        self.geometry("1000x800")
        self._create_widgets()
        self._configure_styles()

    def _configure_styles(self):
        self.style = ttk.Style()
        self.style.theme_use('clam')
        self.style.configure('TLabel', font=('Helvetica', 10))
        self.style.configure('TButton', font=('Helvetica', 10, 'bold'), padding=6)
        self.style.configure('Header.TLabel', font=('Helvetica', 14, 'bold'))

    def _create_widgets(self):
        main_frame = ttk.Frame(self)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        # En-tête
        header_frame = ttk.Frame(main_frame)
        header_frame.pack(fill=tk.X, pady=10)
        
        ttk.Button(header_frame, text="Ouvrir Facture", command=self._open_file).pack(side=tk.LEFT)
        self.status_label = ttk.Label(header_frame, foreground="gray")
        self.status_label.pack(side=tk.RIGHT, padx=10)

        # Affichage des résultats
        self.notebook = ttk.Notebook(main_frame)
        self.notebook.pack(fill=tk.BOTH, expand=True)

        # Onglet Résumé
        summary_frame = ttk.Frame(self.notebook)
        self.summary_text = tk.Text(summary_frame, wrap=tk.WORD, font=('Consolas', 10))
        self.summary_text.pack(fill=tk.BOTH, expand=True)
        self.notebook.add(summary_frame, text="Résumé")

        # Onglet Articles
        self.items_tree = ttk.Treeview(self.notebook, columns=('description', 'quantity', 'price', 'total'), show='headings')
        self.items_tree.heading('description', text='Description')
        self.items_tree.heading('quantity', text='Quantité')
        self.items_tree.heading('price', text='Prix Unitaire')
        self.items_tree.heading('total', text='Total')
        self.notebook.add(self.items_tree, text="Articles")

    def _open_file(self):
        filetypes = [("Fichiers image", "*.png;*.jpg;*.jpeg;*.tiff;*.bmp")]
        filepath = filedialog.askopenfilename(title="Sélectionner une facture", filetypes=filetypes)
        if filepath:
            self.status_label.config(text="Analyse en cours...", foreground="blue")
            threading.Thread(target=self._process_file, args=(filepath,), daemon=True).start()

    def _process_file(self, filepath):
        try:
            processed_img = preprocess_image(filepath)
            text = pytesseract.image_to_string(processed_img)
            
            seller = extract_entity(text, 'Seller')
            client = extract_entity(text, 'Client')
            financials = extract_financial_data(text)
            
            data = {
                'seller': seller,
                'client': client,
                **financials
            }
            
            save_to_database(data)
            self.after(0, self._update_ui, filepath, data)
        except Exception as e:
            self.after(0, self._show_error, str(e))

    def _update_ui(self, filepath, data):
        # Mise à jour de l'onglet Résumé
        summary = [
            f"=== FACTURE ANALYSÉE ===",
            f"Fichier: {filepath}",
            "\n=== VENDEUR ===",
            f"Nom: {data['seller']['name']}",
            f"Adresse:\n{data['seller']['address']}",
            f"Tax ID: {data['seller']['tax_id']}",
            "\n=== CLIENT ===",
            f"Nom: {data['client']['name']}",
            f"Adresse:\n{data['client']['address']}",
            f"Tax ID: {data['client']['tax_id']}",
            "\n=== INFORMATIONS ===",
            f"Numéro: {data['invoice_number']}",
            f"Date: {data['date']}",
            f"IBAN: {data['iban']}",
            f"Total Net: {data['total_net']:.2f}€",
            f"TVA: {data['total_vat']:.2f}€",
            f"Total TTC: {data['total_gross']:.2f}€"
        ]
        self.summary_text.delete(1.0, tk.END)
        self.summary_text.insert(tk.END, '\n'.join(summary))

        # Mise à jour de l'onglet Articles
        self.items_tree.delete(*self.items_tree.get_children())
        for item in data['items']:
            self.items_tree.insert('', 'end', values=(
                item['description'],
                item['quantity'],
                f"{item['unit_price']:.2f}€",
                f"{item['gross_price']:.2f}€"
            ))

        self.status_label.config(text="Analyse réussie", foreground="green")

    def _show_error(self, message):
        messagebox.showerror("Erreur", message)
        self.status_label.config(text="Échec de l'analyse", foreground="red")

def create_tables():
    conn = create_connection()
    if conn:
        cursor = conn.cursor()
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS sellers (
                id INT AUTO_INCREMENT PRIMARY KEY,
                name VARCHAR(255) NOT NULL,
                address TEXT,
                tax_id VARCHAR(255) UNIQUE
            )
        """)
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS clients (
                id INT AUTO_INCREMENT PRIMARY KEY,
                name VARCHAR(255) NOT NULL,
                address TEXT,
                tax_id VARCHAR(255) UNIQUE
            )
        """)
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS invoices (
                id INT AUTO_INCREMENT PRIMARY KEY,
                invoice_number VARCHAR(255),
                date DATE,
                iban VARCHAR(34),
                total_net DECIMAL(10,2),
                total_vat DECIMAL(10,2),
                total_gross DECIMAL(10,2),
                seller_id INT,
                client_id INT,
                FOREIGN KEY (seller_id) REFERENCES sellers(id),
                FOREIGN KEY (client_id) REFERENCES clients(id)
            )
        """)
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS items (
                id INT AUTO_INCREMENT PRIMARY KEY,
                invoice_id INT,
                item_number VARCHAR(50),
                description TEXT,
                quantity DECIMAL(10,2),
                unit_price DECIMAL(10,2),
                net_price DECIMAL(10,2),
                vat_percent DECIMAL(5,2),
                gross_price DECIMAL(10,2),
                FOREIGN KEY (invoice_id) REFERENCES invoices(id)
            )
        """)
        conn.commit()
        cursor.close()
        conn.close()

if __name__ == "__main__":
    create_tables()
    app = InvoiceAnalyzerApp()
    app.mainloop()

In [10]:
import tkinter as tk
import cv2
import pytesseract
import re
import mysql.connector
from datetime import datetime
from tkinter import filedialog, messagebox, ttk
import threading
from mysql.connector import Error

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': 'ayaDATA2025@',
    'database': 'extraction_factures'
}

def preprocess_image(image_path):
    try:
        img = cv2.imread(image_path)
        if img is None:
            raise ValueError("Fichier image introuvable ou corrompu")
        
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        denoised = cv2.fastNlMeansDenoising(gray, h=10)
        _, binarized = cv2.threshold(denoised, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        return binarized
    except Exception as e:
        raise RuntimeError(f"Erreur de prétraitement : {str(e)}")

def extract_entity(text, entity_type):
    pattern = fr'(?s)## {entity_type}:[\s]*\n(.*?)(?=\n##|\nTax Id:|\nIBAN:|\Z)'
    match = re.search(pattern, text, re.IGNORECASE)
    
    entity_info = {'name': '', 'address': '', 'tax_id': ''}
    
    if match:
        content = match.group(1).strip()
        lines = [line.strip() for line in content.split('\n') if line.strip()]
        
        if lines:
            # Extraction du nom (regroupe les lignes jusqu'à l'adresse)
            name_lines = []
            for line in lines:
                if re.match(r'^[A-Za-z0-9\s,&-]+$', line):
                    name_lines.append(line)
                else:
                    break
            entity_info['name'] = ' '.join(name_lines)
            
            # Extraction adresse (lignes restantes)
            address_lines = []
            for line in lines[len(name_lines):]:
                if any(c.isalnum() for c in line) and not line.startswith('Tax Id:'):
                    address_lines.append(line)
            entity_info['address'] = '\n'.join(address_lines)
    
    # Extraction Tax ID avec validation
    tax_match = re.search(fr'Tax Id:\s*([A-Z0-9-]{{8,15}})', text, re.IGNORECASE)
    if tax_match:
        entity_info['tax_id'] = tax_match.group(1)
    
    return entity_info

def extract_financial_data(text):
    data = {
        'invoice_number': '',
        'date': None,
        'iban': '',
        'total_net': 0.0,
        'total_vat': 0.0,
        'total_gross': 0.0,
        'items': []
    }

    # Numéro de facture avec validation renforcée
    inv_match = re.search(r'(Invoice no:|Facture n°?)\s*([A-Z0-9-]{6,})', text, re.IGNORECASE)
    if inv_match:
        data['invoice_number'] = inv_match.group(2).strip()

    # Extraction date multi-formats avec validation
    date_match = re.search(r'(Date of issue|Date):?\s*(\d{1,2}[\/\-\.]\d{1,2}[\/\-\.]\d{2,4})', text, re.IGNORECASE)
    if date_match:
        raw_date = date_match.group(2)
        for fmt in ('%m/%d/%Y', '%d/%m/%Y', '%m-%d-%Y', '%d-%m-%Y', '%Y%m%d'):
            try:
                data['date'] = datetime.strptime(raw_date, fmt).strftime('%Y-%m-%d')
                break
            except ValueError:
                continue

    # Extraction IBAN robuste avec validation
    iban_match = re.search(r'IBAN:\s*([A-Z]{2}\d{2}[\sA-Z0-9]{10,30})', text)
    if iban_match:
        data['iban'] = iban_match.group(1).replace(' ', '')

    # Nouveau système d'extraction des articles
    items_section = re.search(r'## ITEMS([\s\S]*?)## SUMMARY', text)
    if items_section:
        items_text = items_section.group(1)
        item_pattern = (
            r'(\d+\.)\s+'                      # Numéro article
            r'((?:.(?!\d+[\.,]\d+\s+each))+)\s+' # Description
            r'(\d+[\.,]\d+)\s+each\s+'         # Quantité
            r'([\d\s\.,]+)\s+'                 # Prix unitaire
            r'([\d\s\.,]+)\s+'                 # Valeur nette
            r'(\d+%)\s+'                       # TVA
            r'([\d\s\.,]+)'                    # Valeur brute
        )
        items = re.findall(item_pattern, items_text)
        
        for item in items:
            description = re.sub(r'\s+', ' ', item[1]).strip()
            data['items'].append({
                'number': item[0].strip('.'),
                'description': description,
                'quantity': float(item[2].replace(',', '.')),
                'unit_price': parse_amount(item[3]),
                'net_price': parse_amount(item[4]),
                'vat_percent': float(item[5].strip('%')),
                'gross_price': parse_amount(item[6])
            })

    # Calcul des totaux basé sur les articles + vérification
    if data['items']:
        data['total_net'] = sum(item['net_price'] for item in data['items'])
        data['total_vat'] = sum(item['gross_price'] - item['net_price'] for item in data['items'])
        data['total_gross'] = sum(item['gross_price'] for item in data['items'])
    else:
        # Fallback sur l'extraction des totaux
        total_pattern = r'Total\s*\|\s*([\d\s\.,]+)\s*\|\s*([\d\s\.,]+)\s*\|\s*([\d\s\.,]+)'
        totals_match = re.search(total_pattern, text.replace('\n', ''))
        if totals_match:
            data['total_net'] = parse_amount(totals_match.group(1))
            data['total_vat'] = parse_amount(totals_match.group(2))
            data['total_gross'] = parse_amount(totals_match.group(3))

    return data

def parse_amount(amount_str):
    # Gestion avancée des formats numériques
    clean_str = re.sub(r'[^\d,.]', '', amount_str)
    
    if ',' in clean_str and '.' in clean_str:
        # Format européen : 1.234,56
        return float(clean_str.replace('.', '').replace(',', '.'))
    elif ',' in clean_str and len(clean_str.split(',')[-1]) == 2:
        # Format français : 1,234.56
        return float(clean_str.replace(',', ''))
    else:
        # Format international
        return float(clean_str.replace(',', ''))

def create_connection():
    try:
        return mysql.connector.connect(**DB_CONFIG)
    except Error as e:
        messagebox.showerror("Erreur Base de Données", f"Connexion impossible : {str(e)}")
        return None

def save_to_database(data):
    conn = create_connection()
    if not conn:
        return

    try:
        cursor = conn.cursor()

        # Insertion des entités
        entities = ['seller', 'client']
        for entity in entities:
            cursor.execute(f"""
                INSERT INTO {entity}s (name, address, tax_id)
                VALUES (%s, %s, %s)
                ON DUPLICATE KEY UPDATE
                name = VALUES(name),
                address = VALUES(address)
            """, (
                data[entity]['name'],
                data[entity]['address'],
                data[entity]['tax_id']
            ))

        # Insertion de la facture
        cursor.execute("""
            INSERT INTO invoices (
                invoice_number, date, iban,
                total_net, total_vat, total_gross,
                seller_id, client_id
            ) VALUES (
                %s, %s, %s,
                %s, %s, %s,
                (SELECT id FROM sellers WHERE tax_id = %s),
                (SELECT id FROM clients WHERE tax_id = %s)
            )
        """, (
            data['invoice_number'],
            data['date'],
            data['iban'],
            data['total_net'],
            data['total_vat'],
            data['total_gross'],
            data['seller']['tax_id'],
            data['client']['tax_id']
        ))

        # Insertion des articles
        invoice_id = cursor.lastrowid
        for item in data['items']:
            cursor.execute("""
                INSERT INTO items (
                    invoice_id, item_number, description,
                    quantity, unit_price, net_price,
                    vat_percent, gross_price
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """, (
                invoice_id,
                item['number'],
                item['description'],
                item['quantity'],
                item['unit_price'],
                item['net_price'],
                item['vat_percent'],
                item['gross_price']
            ))

        conn.commit()
    except Error as e:
        conn.rollback()
        raise RuntimeError(f"Erreur base de données : {str(e)}")
    finally:
        cursor.close()
        conn.close()

class InvoiceAnalyzerApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("InvoiceMaster Pro")
        self.geometry("1200x800")
        self._create_widgets()
        self._configure_styles()

    def _configure_styles(self):
        self.style = ttk.Style()
        self.style.theme_use('clam')
        self.style.configure('TLabel', font=('Segoe UI', 10))
        self.style.configure('TButton', font=('Segoe UI', 10, 'bold'), padding=6)
        self.style.configure('Treeview', rowheight=25)

    def _create_widgets(self):
        main_frame = ttk.Frame(self)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        # Header
        header_frame = ttk.Frame(main_frame)
        header_frame.pack(fill=tk.X, pady=10)
        
        ttk.Button(header_frame, 
                 text="Ouvrir Facture", 
                 command=self._open_file).pack(side=tk.LEFT)
        
        self.status_label = ttk.Label(header_frame, foreground="gray")
        self.status_label.pack(side=tk.RIGHT, padx=10)

        # Notebook
        self.notebook = ttk.Notebook(main_frame)
        self.notebook.pack(fill=tk.BOTH, expand=True)

        # Onglet Résumé
        summary_frame = ttk.Frame(self.notebook)
        self.summary_text = tk.Text(summary_frame, wrap=tk.WORD, font=('Consolas', 10))
        self.summary_text.pack(fill=tk.BOTH, expand=True)
        self.notebook.add(summary_frame, text="Résumé")

        # Onglet Articles
        items_frame = ttk.Frame(self.notebook)
        self.items_tree = ttk.Treeview(items_frame, 
                                     columns=('no', 'desc', 'qty', 'price', 'total'), 
                                     show='headings')
        self.items_tree.heading('no', text='N°')
        self.items_tree.heading('desc', text='Description')
        self.items_tree.heading('qty', text='Quantité')
        self.items_tree.heading('price', text='Prix Unitaire')
        self.items_tree.heading('total', text='Total')
        
        vsb = ttk.Scrollbar(items_frame, orient="vertical", command=self.items_tree.yview)
        self.items_tree.configure(yscrollcommand=vsb.set)
        
        self.items_tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        vsb.pack(side=tk.RIGHT, fill=tk.Y)
        
        self.notebook.add(items_frame, text="Articles")

    def _open_file(self):
        filetypes = [("Fichiers image", "*.png;*.jpg;*.jpeg;*.tiff;*.bmp")]
        filepath = filedialog.askopenfilename(title="Sélectionner une facture", 
                                            filetypes=filetypes)
        if filepath:
            self.status_label.config(text="Analyse en cours...", foreground="blue")
            threading.Thread(target=self._process_file, 
                           args=(filepath,), 
                           daemon=True).start()

    def _process_file(self, filepath):
        try:
            processed_img = preprocess_image(filepath)
            text = pytesseract.image_to_string(processed_img)
            
            seller = extract_entity(text, 'Seller')
            client = extract_entity(text, 'Client')
            financials = extract_financial_data(text)
            
            data = {
                'seller': seller,
                'client': client,
                **financials
            }
            
            save_to_database(data)
            self.after(0, self._update_ui, filepath, data)
        except Exception as e:
            self.after(0, self._show_error, str(e))

    def _update_ui(self, filepath, data):
        # Mise à jour du résumé
        summary_content = [
            f"=== FACTURE ANALYSÉE ===",
            f"Fichier: {filepath}",
            "\n=== VENDEUR ===",
            f"Nom: {data['seller']['name']}" if data['seller']['name'] else "Nom: Non détecté",
            f"Adresse:\n{data['seller']['address']}" if data['seller']['address'] else "Adresse: Non détectée",
            f"Tax ID: {data['seller']['tax_id']}",
            "\n=== CLIENT ===",
            f"Nom: {data['client']['name']}" if data['client']['name'] else "Nom: Non détecté",
            f"Adresse:\n{data['client']['address']}" if data['client']['address'] else "Adresse: Non détectée",
            f"Tax ID: {data['client']['tax_id']}",
            "\n=== INFORMATIONS ===",
            f"Numéro: {data['invoice_number']}",
            f"Date: {data['date']}",
            f"IBAN: {data['iban']}",
            f"Total Net: {data['total_net']:.2f}€",
            f"TVA: {data['total_vat']:.2f}€",
            f"Total TTC: {data['total_gross']:.2f}€"
        ]
        self.summary_text.delete(1.0, tk.END)
        self.summary_text.insert(tk.END, '\n'.join(summary_content))

        # Mise à jour des articles
        self.items_tree.delete(*self.items_tree.get_children())
        for item in data['items']:
            self.items_tree.insert('', 'end', values=(
                item['number'],
                item['description'],
                f"{item['quantity']:.2f}",
                f"{item['unit_price']:.2f}€",
                f"{item['gross_price']:.2f}€"
            ))

        self.status_label.config(text="Analyse réussie", foreground="green")

    def _show_error(self, message):
        messagebox.showerror("Erreur", message)
        self.status_label.config(text="Échec de l'analyse", foreground="red")
        self.summary_text.delete(1.0, tk.END)
        self.items_tree.delete(*self.items_tree.get_children())

def create_tables():
    conn = create_connection()
    if conn:
        cursor = conn.cursor()
        
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS sellers (
                id INT AUTO_INCREMENT PRIMARY KEY,
                name VARCHAR(255) NOT NULL,
                address TEXT,
                tax_id VARCHAR(255) UNIQUE
            )
        """)
        
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS clients (
                id INT AUTO_INCREMENT PRIMARY KEY,
                name VARCHAR(255) NOT NULL,
                address TEXT,
                tax_id VARCHAR(255) UNIQUE
            )
        """)
        
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS invoices (
                id INT AUTO_INCREMENT PRIMARY KEY,
                invoice_number VARCHAR(255),
                date DATE,
                iban VARCHAR(34),
                total_net DECIMAL(10,2),
                total_vat DECIMAL(10,2),
                total_gross DECIMAL(10,2),
                seller_id INT,
                client_id INT,
                FOREIGN KEY (seller_id) REFERENCES sellers(id),
                FOREIGN KEY (client_id) REFERENCES clients(id)
            )
        """)
        
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS items (
                id INT AUTO_INCREMENT PRIMARY KEY,
                invoice_id INT,
                item_number VARCHAR(50),
                description TEXT,
                quantity DECIMAL(10,2),
                unit_price DECIMAL(10,2),
                net_price DECIMAL(10,2),
                vat_percent DECIMAL(5,2),
                gross_price DECIMAL(10,2),
                FOREIGN KEY (invoice_id) REFERENCES invoices(id)
            )
        """)
        
        conn.commit()
        cursor.close()
        conn.close()

if __name__ == "__main__":
    create_tables()
    app = InvoiceAnalyzerApp()
    app.mainloop()

In [19]:
import tkinter as tk
import cv2
import pytesseract
import re
import mysql.connector
from PIL import Image
from tkinter import filedialog, messagebox, ttk
import threading
from datetime import datetime  # Ajout de l'import

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': 'ayaDATA2025@',
    'database': 'extraction_factures'
}

def preprocess_image(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    binarized = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    denoised = cv2.medianBlur(binarized, 3)
    return denoised

def extract_entity(text, entity_type):
    pattern = fr'(?s)## {entity_type}:[\s]*(.*?)(?=\n##|\nTax Id:|\nIBAN:|\Z)'
    match = re.search(pattern, text, re.IGNORECASE)
    
    entity_info = {'name': '', 'address': '', 'tax_id': ''}
    
    if match:
        content = match.group(1).strip()
        lines = [line.strip() for line in content.split('\n') if line.strip()]
        
        if lines:
            entity_info['name'] = lines[0]
            address_lines = []
            for line in lines[1:]:
                if any(c.isalnum() for c in line) and not line.startswith('Tax Id:'):
                    address_lines.append(line)
            entity_info['address'] = '\n'.join(address_lines)
    
    tax_match = re.search(fr'Tax Id:\s*(\d{{3}}-\d{{2}}-\d{{4}})', text, re.IGNORECASE)
    if tax_match:
        entity_info['tax_id'] = tax_match.group(1)
    
    return entity_info

def extract_financial_data(text):
    totals = {'net': 0.0, 'vat': 0.0, 'gross': 0.0}
    
    total_pattern = r'Total\s+([\d\s.,]+)\s+([\d\s.,]+)\s+([\d\s.,]+)'
    totals_match = re.search(total_pattern, text.replace('\n', ''))
    
    if totals_match:
        totals['gross'] = float(totals_match.group(3).replace(' ', '').replace(',', '.'))
        totals['vat'] = float(totals_match.group(2).replace(' ', '').replace(',', '.'))
        totals['net'] = float(totals_match.group(1).replace(' ', '').replace(',', '.'))
    else:
        amounts = re.findall(r'\b\d{1,3}(?:[\s.]\d{3})*(?:,\d{2})?\b', text)
        clean_amounts = [float(a.replace(' ', '').replace(',', '.')) for a in amounts if float(a.replace(' ', '').replace(',', '.')) > 10]
        if clean_amounts:
            totals['gross'] = max(clean_amounts)
            totals['vat'] = totals['gross'] * 0.1
            totals['net'] = totals['gross'] - totals['vat']

    return totals

def analyse_facture(filepath):
    try:
        processed_img = preprocess_image(filepath)
        text = pytesseract.image_to_string(processed_img)
        text = re.sub(r'Tota[l1i]', 'Total', text, flags=re.IGNORECASE)

        seller = extract_entity(text, 'Seller')
        client = extract_entity(text, 'Client')
        totals = extract_financial_data(text)

        invoice_number = re.search(r'Invoice no[:\s]*([A-Z0-9-]{6,})', text)
        date_match = re.search(r'\b(\d{2}[/-]\d{2}[/-]\d{4})\b', text)
        iban = re.search(r'IBAN[:\s]*([A-Z]{2}\d{2}[\sA-Z0-9]{10,30})', text)

        # Conversion de la date au format MySQL
        mysql_date = None
        if date_match:
            try:
                date_str = date_match.group(1).replace('/', '-')
                date_obj = datetime.strptime(date_str, '%d-%m-%Y')
                mysql_date = date_obj.strftime('%Y-%m-%d')
            except ValueError:
                pass

        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor()

        def get_or_create_id(table, data):
            cursor.execute(f"SELECT id FROM {table} WHERE tax_id = %s", (data['tax_id'],))
            result = cursor.fetchone()
            if result:
                return result[0]
            cursor.execute(f"""
                INSERT INTO {table} (name, address, tax_id)
                VALUES (%s, %s, %s)
            """, (data['name'], data['address'], data['tax_id']))
            return cursor.lastrowid

        seller_id = get_or_create_id('sellers', seller)
        client_id = get_or_create_id('clients', client)

        cursor.execute("""
            INSERT INTO invoices (invoice_number, date, iban, total_net, total_vat, total_gross, seller_id, client_id)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            invoice_number.group(1) if invoice_number else '',
            mysql_date,
            iban.group(1) if iban else '',
            totals['net'],
            totals['vat'],
            totals['gross'],
            seller_id,
            client_id
        ))

        conn.commit()
        cursor.close()
        conn.close()

        return {
            'seller': seller,
            'client': client,
            'invoice_number': invoice_number.group(1) if invoice_number else '',
            'date': date_match.group(1) if date_match else '',
            'iban': iban.group(1) if iban else '',
            'totals': totals
        }

    except Exception as e:
        raise RuntimeError(f"Erreur: {str(e)}")

class InvoiceAnalyzerApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Analyseur de Factures")
        self.geometry("800x600")
        self._create_widgets()
        self._configure_styles()

    def _configure_styles(self):
        self.style = ttk.Style()
        self.style.theme_use('clam')
        self.style.configure('TLabel', font=('Segoe UI', 10))
        self.style.configure('TButton', font=('Segoe UI', 10, 'bold'), padding=6)

    def _create_widgets(self):
        main_frame = ttk.Frame(self)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        header_frame = ttk.Frame(main_frame)
        header_frame.pack(fill=tk.X, pady=10)
        
        ttk.Button(header_frame, text="Ouvrir Facture", command=self._open_file).pack(side=tk.LEFT)
        self.status_label = ttk.Label(header_frame, foreground="gray")
        self.status_label.pack(side=tk.RIGHT, padx=10)

        self.notebook = ttk.Notebook(main_frame)
        self.notebook.pack(fill=tk.BOTH, expand=True)

        summary_frame = ttk.Frame(self.notebook)
        self.summary_text = tk.Text(summary_frame, wrap=tk.WORD, font=('Consolas', 10))
        self.summary_text.pack(fill=tk.BOTH, expand=True)
        self.notebook.add(summary_frame, text="Résumé")

    def _open_file(self):
        filetypes = [("Images", "*.png;*.jpg;*.jpeg;*.tiff;*.bmp")]
        filepath = filedialog.askopenfilename(title="Sélectionner une facture", filetypes=filetypes)
        if filepath:
            self.status_label.config(text="Analyse en cours...", foreground="blue")
            threading.Thread(target=self._process_file, args=(filepath,), daemon=True).start()

    def _process_file(self, filepath):
        try:
            data = analyse_facture(filepath)
            self.after(0, self._update_ui, filepath, data)
        except Exception as e:
            self.after(0, self._show_error, str(e))

    def _update_ui(self, filepath, data):
        summary_content = [
            f"=== FACTURE ANALYSÉE ===",
            f"Fichier: {filepath}",
            "\n=== VENDEUR ===",
            f"Nom: {data['seller']['name'] or 'Non détecté'}",
            f"Adresse:\n{data['seller']['address'] or 'Non détectée'}",
            f"Tax ID: {data['seller']['tax_id']}",
            "\n=== CLIENT ===",
            f"Nom: {data['client']['name'] or 'Non détecté'}",
            f"Adresse:\n{data['client']['address'] or 'Non détectée'}",
            f"Tax ID: {data['client']['tax_id']}",
            "\n=== INFORMATIONS ===",
            f"Numéro: {data['invoice_number']}",
            f"Date: {data['date']}",
            f"IBAN: {data['iban']}",
            "\n=== TOTAUX ===",
            f"Total Net: {data['totals']['net']:.2f}€",
            f"TVA: {data['totals']['vat']:.2f}€",
            f"Total TTC: {data['totals']['gross']:.2f}€"
        ]
        
        self.summary_text.delete(1.0, tk.END)
        self.summary_text.insert(tk.END, '\n'.join(summary_content))
        self.status_label.config(text="Analyse réussie", foreground="green")

    def _show_error(self, message):
        messagebox.showerror("Erreur", message)
        self.status_label.config(text="Échec de l'analyse", foreground="red")

def create_tables():
    conn = mysql.connector.connect(**DB_CONFIG)
    cursor = conn.cursor()
    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS sellers (
            id INT AUTO_INCREMENT PRIMARY KEY,
            name VARCHAR(255) NOT NULL,
            address TEXT,
            tax_id VARCHAR(255) UNIQUE
        )
    """)
    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS clients (
            id INT AUTO_INCREMENT PRIMARY KEY,
            name VARCHAR(255) NOT NULL,
            address TEXT,
            tax_id VARCHAR(255) UNIQUE
        )
    """)
    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS invoices (
            id INT AUTO_INCREMENT PRIMARY KEY,
            invoice_number VARCHAR(255),
            date DATE,
            iban VARCHAR(34),
            total_net DECIMAL(10,2),
            total_vat DECIMAL(10,2),
            total_gross DECIMAL(10,2),
            seller_id INT,
            client_id INT,
            FOREIGN KEY (seller_id) REFERENCES sellers(id),
            FOREIGN KEY (client_id) REFERENCES clients(id)
        )
    """)
    
    conn.commit()
    cursor.close()
    conn.close()

if __name__ == "__main__":
    create_tables()
    app = InvoiceAnalyzerApp()
    app.mainloop()